In [8]:
def centering_step(Q, p, A, b, t, v0, eps, verbose=True):
    """
    Newton's method for the centering step in the barrier method.

    Args:
        Q (ndarray): Quadratic term (n x n positive semi-definite matrix).
        p (ndarray): Linear term (n-dimensional vector).
        A (ndarray): Constraint matrix (m x n).
        b (ndarray): Constraint vector (m-dimensional vector).
        t (float): Barrier method parameter.
        v0 (ndarray): Initial feasible point (n-dimensional vector).
        eps (float): Target precision for Newton's method.
        verbose (bool): If True, print debug information.

    Returns:
        list: Sequence of variable iterates (v_i) until convergence.
    """
    def barrier_function(v):
        """
        Objective function with barrier term.
        """
        barrier_term = -np.sum(np.log(b - A @ v))
        return t * (0.5 * v.T @ Q @ v + p.T @ v) + barrier_term

    def barrier_gradient(v):
        """
        Gradient of the barrier function.
        """
        grad = t * (Q @ v + p) + np.sum(A.T / (b - A @ v)[:, np.newaxis], axis=1)
        return grad

    def barrier_hessian(v):
        """
        Hessian of the barrier function.
        """
        H = t * Q
        for i in range(len(b)):
            ai = A[i, :]
            H += np.outer(ai, ai) / (b[i] - ai @ v)**2
        return H

    def backtracking_line_search(v, delta_v, grad, barrier_function, alpha=0.1, beta=0.7):
        """
        Backtracking line search to find the step size t.
        """
        t = 1
        while True:
            new_v = v + t * delta_v
            if np.any(b - A @ new_v <= 0):  # Feasibility check
                t *= beta
                if verbose:
                    print(f"  Backtracking: Feasibility violated, reducing t to {t:.6f}")
                continue
            
            lhs = barrier_function(new_v)
            rhs = barrier_function(v) + alpha * t * grad.T @ delta_v
            
            if lhs < rhs:
                if verbose:
                    print(f"  Backtracking: t = {t:.6f}, lhs = {lhs:.6f}, rhs = {rhs:.6f} (Condition satisfied)")
                break
            
            t *= beta
            if verbose:
                print(f"  Backtracking: t = {t:.6f}, lhs = {lhs:.6f}, rhs = {rhs:.6f} (Condition not satisfied)")
            
            if t < 1e-12:
                raise RuntimeError("Backtracking line search failed: step size too small.")
        
        return t

    # Initialize variables
    v = v0
    sequence = [v]
    iteration = 0

    while True:
        iteration += 1
        grad = barrier_gradient(v)
        hess = barrier_hessian(v)
        
        try:
            delta_v = -np.linalg.solve(hess, grad)  # Newton direction
        except np.linalg.LinAlgError as e:
            raise RuntimeError(f"Hessian inversion failed at iteration {iteration}: {e}")

        # Log gradient and direction
        if verbose:
            print(f"Iteration {iteration}:")
            print(f"  Gradient norm: {np.linalg.norm(grad):.6f}")
            print(f"  Newton direction norm: {np.linalg.norm(delta_v):.6f}")
            print(f"  Feasibility check: {np.min(b - A @ v):.6f}")

        # Check if the direction is descent
        if grad.T @ delta_v >= 0:
            raise RuntimeError(f"Direction is not descent: grad^T delta_v = {grad.T @ delta_v:.6f}")

        # Stopping criterion
        if np.linalg.norm(delta_v) <= eps:
            if verbose:
                print("  Converged!")
            break

        # Perform backtracking line search
        step_size = backtracking_line_search(v, delta_v, grad, barrier_function)
        v = v + step_size * delta_v
        sequence.append(v)

        # Safety condition to avoid infinite loops
        if iteration > 1000:
            raise RuntimeError("Centering step did not converge within 1000 iterations.")

    return sequence


In [3]:
def barr_method(Q, p, A, b, v0, eps, mu=10):
    """
    Barrier method to solve the QP problem.

    Args:
        Q (ndarray): Quadratic term (n x n positive semi-definite matrix).
        p (ndarray): Linear term (n-dimensional vector).
        A (ndarray): Constraint matrix (m x n).
        b (ndarray): Constraint vector (m-dimensional vector).
        v0 (ndarray): Initial feasible point (n-dimensional vector).
        eps (float): Target precision for the overall barrier method.
        mu (float): Scaling factor for the barrier parameter t.

    Returns:
        list: Sequence of variable iterates (v_i) until convergence.
    """
    def objective_function(v):
        """
        Objective function of the original QP problem.
        """
        return 0.5 * v.T @ Q @ v + p.T @ v

    m = len(b)  # Number of constraints
    t = 1  # Initial barrier parameter
    v = v0
    sequence = [v]

    while True:
        # Solve the centering step for the current t
        centering_sequence = centering_step(Q, p, A, b, t, v, eps)
        v = centering_sequence[-1]  # Take the final value from centering step
        sequence.extend(centering_sequence)

        # Check duality gap
        gap = m / t
        if gap < eps:
            break

        # Increase the barrier parameter
        t *= mu

    return sequence


In [ ]:
# Random data
np.random.seed(42)
n, d = 100, 20  # Number of samples and features
X = np.random.randn(n, d)
y = np.random.randn(n)
lambda_val = 10

# Problems parameters
Q = X.T @ X
p = -X.T @ y
A = np.vstack([np.eye(d), -np.eye(d)])
b = np.hstack([lambda_val * np.ones(d), lambda_val * np.ones(d)])

# Initialization
v0 = np.zeros(d)  # Initial w is zero (feasible)
eps = 1e-6  # Precision criterion

# Barrier method with different mu values
mu_values = [2, 15, 50, 100]
results = {}

for mu in mu_values:
    print(f"Running barrier method with mu = {mu}")
    sequence = barr_method(Q, p, A, b, v0, eps, mu)
    results[mu] = sequence

# Compute f(v_t) for each iteration
def objective_function(v):
    return 0.5 * v.T @ Q @ v + p.T @ v

# Use the best final value found for f*
f_star = min(objective_function(sequence[-1]) for sequence in results.values())

# Plotting
plt.figure(figsize=(10, 6))
for mu, sequence in results.items():
    f_values = [objective_function(v) for v in sequence]
    gap = [f - f_star for f in f_values]
    plt.semilogy(gap, label=f"mu = {mu}")

plt.xlabel("Iterations")
plt.ylabel("Gap: $f(v_t) - f^*$ (log scale)")
plt.title("Barrier Method Convergence")
plt.legend()
plt.grid(True)
plt.show()


final_solutions = {}

for mu, sequence in results.items():
    final_solutions[mu] = sequence[-1]  # Store the last iterate (final w)

# Visualize w for each mu
plt.figure(figsize=(10, 6))
bar_width = 0.2  # Width of each bar group

indices = np.arange(len(final_solutions[2]))  # Indices of w (0, 1, ..., d-1)
for i, (mu, w_star) in enumerate(final_solutions.items()):
    plt.bar(indices + i * bar_width, w_star, bar_width, label=f"mu = {mu}")

# Labels and title
plt.xlabel("w components")
plt.ylabel("w components' value")
plt.title("Final w for different values of mu")
plt.xticks(indices + bar_width * (len(final_solutions) - 1) / 2, indices)  # Center ticks
plt.legend()
plt.grid(axis="y")
plt.show()

Running barrier method with mu = 2


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 20 is different from 100)